In [9]:
# Ensure you have the following installed:
# numpy
# os
# compas
# compas_fd
# warnings
# traceback
# pickle

# Install by running:
# !pip install -r requirements.txt

In [10]:
import numpy as np
import pickle
import pandas as pd

# G-code generation codes are cloned from https://github.com/tibor-barsi/GcodeGenerator. Tibor Barsi is the author of the code, and was a PhD student at the Ladisk lab of the University of Ljubljana
#  We should be careful with crediting the author of the code, if we ever want to make these code public.
from src.g_code_generation_copy.gcode_generator import G_code_generator
from src.g_code_generation_copy.tool_changer_functions import save_params, load_params, printer_start, load_tool, unload_tool, tool_change, take_photo, play_sound, printer_stop
# from src.additional_functions import *
from src.network import Network_custom, replace_brackets
import os

BYU_UW_root = r"G:\.shortcut-targets-by-id\1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx\Illimited Lab Projects\Research Projects\Spiders\BYU-UW"

### Step 1: Define the network

In [ ]:
# Note that the coordinates of the vertices will change after equilibrium is reached.
vertices = np.array([
    [10, 50, 0], [10, 45, 0], [12, 52, 0], [12, 43, 0], [15, 60, 0], [14, 41, 0], [15, 42, 0], 
    [20, 51, 0], [22, 60, 0], [16, 62, 0], [22, 65, 0], [23, 64, 0], [25, 63, 0], [25, 58, 0], 
    [24, 55, 0], [23, 43, 0], [17, 35, 0], [26, 37, 0], [18, 33, 0], [30, 25, 0], [33, 27, 0], 
    [35, 29, 0], [37, 35, 0], [33, 42, 0], [33, 46, 0], [33, 50, 0], [34, 52, 0], [40, 60, 0], 
    [41, 52, 0], [41, 48, 0], [40, 48, 0], [39, 47, 0], [38, 46, 0], [37, 45, 0], [37, 44, 0], 
    [36, 43, 0], [36, 42, 0], [37, 41, 0], [38, 40, 0], [39, 39, 0], [40, 38, 0], [41, 38, 0], 
    [42, 32, 0], [43, 29, 0], [50, 58, 0], [48, 50, 0], [46, 48, 0], [47, 47, 0], [48, 46, 0], 
    [49, 45, 0], [49, 44, 0], [48, 43, 0], [47, 41, 0], [51, 57, 0], [53, 55, 0], [51, 53, 0], 
    [52, 53, 0], [52, 46, 0], [54, 50, 0], [54, 49, 0], [51, 43, 0], [8, 47, 0], [22, 21, 0], 
    [60, 40, 0], [59, 60, 0], [18, 68, 0], [44, 42, 0]
])

# Define edges with paths
edges = [
    [1, 2], [3, 1], [1, 62], [62, 2], [2, 4], [4, 6], [6, 17], [17, 19], 
    [19, 63], [63, 20], [20, 21], [21, 22], [22, 44], [44, 61], [61, 64], 
    [64, 60], [60, 59], [59, 57], [57, 55], [55, 65], [65, 54], [54, 45], 
    [45, 28], [28, 13], [13, 12], [12, 11], [11, 66], [66, 10], [10, 5], 
    [5, 3], [12, 9], [9, 8], [8, 7], [7, 6], [5, 9], [9, 14], [14, 27], 
    [27, 32], [3, 8], [8, 15], [15, 26], [26, 34], [4, 7], [7, 16], 
    [16, 25], [25, 36], [17, 18], [18, 24], [24, 38], [13, 14], [14, 15], 
    [15, 16], [16, 18], [18, 21], [22, 23], [23, 40], [44, 43], [43, 42], 
    [61, 60], [28, 29], [29, 30], [45, 46], [46, 48], [54, 55], [19, 20], 
    [10, 11], [29, 46], [46, 56], [56, 58], [58, 43], [43, 23], [23, 24], 
    [24, 25], [25, 26], [26, 27], [27, 29], [35, 34], [34, 33], [33, 32], [32, 31], 
    [31, 30], [30, 47], [47, 48], [48, 49], [49, 50], [50, 51], [51, 52], 
    [52, 53], [53, 42], [42, 41], [41, 40], [40, 39], [39, 38], [38, 37], [37, 36], 
    [36, 35], [37, 67], [67, 39], [41, 67], [67, 53], [51, 67], [67, 49], 
    [47, 67], [67, 31], [33, 67], [67, 35], [52, 58], [58, 59], [50, 56], 
    [56, 57]
]
adjusted_edges = [(e[0] - 1, e[1] - 1) for e in edges]
paths       = [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11]]   # 3D printable paths. Ensure an edge is not flipped.
directions  = np.ones(len(edges))          # Flip the curvature of an edge by setting the direction to -1.

q           = np.ones(len(edges)) * 0.1
fixed       = [62, 63, 64, 65, 66]
# model_name  = 'VS_ring{}_connectors{}_center{}_as{}'.format(q_ring, q_connectors, q_center, q[2]*scalar)
# Create the network
net = Network_custom.from_fd(vertices, adjusted_edges, q, fixed, paths = None, dir = directions)

# The topology of structure is fixed after generation, but you can update force densities after making the network
# q[0] = .10
# net.update_shape(q)

# Plot the network like this
net.net_plot(color=True, elables = False, vlabels = True)

(net.vertices == vertices).all() # Please understand that this is False
np.max(net.f)/0.078294515

12.68976491427033

### Step 2: Materialize

In [28]:
%matplotlib qt
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot vertices
ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], c='r', marker='o')

# Plot edges
for edge in adjusted_edges:
    v1, v2 = vertices[edge[0]], vertices[edge[1]]
    ax.plot([v1[0], v2[0]], [v1[1], v2[1]], [v1[2], v2[2]], 'b-')
for i, v in enumerate(vertices):
    ax.text(v[0], v[1], v[2], str(i+1), color='blue')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.show()

In [64]:
# TPUc = {'E':130, 'v':0.3897, 'p':1.18e-9, 'A':0.11*0.71, 'name': 'TPU conductive'} # Conductive TPU. Manufacturer Ninjatek Eel
TPU = {'E':77, 'v':0.3897, 'p':1.18e-9, 'A':0.11*0.71, 'name': 'TPU non-conductive'} # Conductive TPU. Manufacturer Ninjatek Eel

file_path = os.path.join(BYU_UW_root, 'Avg_Stress_Strain_Overture_TPU.csv')
stress_data, strain_data = net.load_stress_strain_curve(file_path)

TPU_nl = {'stress':strain_data, 'strain': stress_data, 'v':0.3897, 'p':1.18e-9, 'A': 0.078294515, 'name': 'TPU Overture'} # TPU Overture non-conductive

net.set_material(TPU) # By saving the material properties in the network, we can easily see how the network was constructed.

# If all elements will get the same material, you can use the following line
# E = [TPU['E']]*len(net.edges)
A = [TPU_nl['A']]*len(net.edges)
l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')
# l0, l_scalar = net.materialize(E, A)
# Yielding the initial lengths of each element. And the following scalar: min(l0/l1)
net.l0, net.l_scalar

(array([66.37705898, 54.17817354, 13.42631463, 63.62056558, 60.0757708 ,
        44.74401457, 45.97906874, 65.02207759, 59.72333052, 50.1583892 ,
        54.634282  , 30.08702505]),
 0.8374694131835344)

In [65]:
# # validate nl material method. This should be the same for the linear TPU
# strain_data = np.linspace(0, 0.5, 100)
# stress_data = TPU['E']*strain_data
# TPU_check = {'stress':strain_data, 'strain': stress_data, 'v':0.3897, 'p':1.18e-9, 'A':0.11*0.71, 'name': 'TPU Check'}

# net.set_material(TPU_check) # By saving the material properties in the network, we can easily see how the network was constructed.
# l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')
# net.l0, net.l_scalar

### Step 3: Define elements as a curves
An (optional) optimization algorithm finds the vertex locations minimizing abs(l1-l0). Next the (optimized) network is scaled down, and edges are defined as arcs.

available error functions in the optimization algorithms are:
- 'no optimization' : Network is scaled down directly, and no alogirthm is applied
- 'standard'        : a standard L2-norm quantifies the error
- 'sigmoid'         : a sigmoid function skews the L2-norm to penalize too short edges more (b>0) or too long edges more (b<0)

Preliminary results shows best results using a standard L2 error function

In [94]:
# net.initialize_shape_optimizer(function_type = 'no optimization',  method = 'L-BFGS-B', params = None)
# net.initialize_shape_optimizer(function_type = 'standard',  method = 'L-BFGS-B',options ={"maxiter": 10000, "maxfun": 1500000, "maxls": 5000})
net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 1000, "damping": .1, "correction_scalar": 1.5, "tol": 1e-6})
# net.initialize_shape_optimizer(function_type = 'standard',  method = 'trust-constr')
# net.initialize_shape_optimizer(function_type = 'sigmoid',  method = 'L-BFGS-B', params = {'a': 4, 'b': -1})
net.optimize_vertices()
reference_point = [0,0,0]                         # The network will be scaler relative to this point
net.scale_vertices(reference_point, net.l_scalar, account_for_leafs = True) # If you don't provide a scalar, it will use network.l_scalar automatically

# #  You can plot the scaled network like this
# # net.net_plot(color=True, plot_type='scaled')

# # Determine the Radius and Angle of the circle that define the arc of each element
R, th = net.arc_param()
# # for the arc length (net.l1 * net.l_scalar) is used unless specified otherwise, for the cord length (net.l0) is used unless specified otherwise
xyz = net.arc_points(n = 10) 

net.net_plot(color=True, plot_type='arcs', elables=True, vlabels = True)
# net.net_plot(color=True, plot_type='optimized', elables=False, vlabels = False)

# np.min(R/net.l0), net.l_scalar, np.min(net.l0 - net.l1), np.max(net.l0 - net.l1)
net.l_scalar

Iteration 0: Current error = 425.6290392320517
Iteration 100: Current error = 2.6731009639958616
Iteration 200: Current error = 2.195167916137413
Iteration 300: Current error = 2.18661552487397
Iteration 400: Current error = 2.186021948149681
Converged after 428 iterations.
Final error: 2.185985360837885


0.9853807673777721

In [67]:
%matplotlib qt
import matplotlib.animation as animation
ani = net.ShapeOptimizer.animate_optimization()
writer = animation.FFMpegWriter(fps=15)
ani.save(os.path.join(BYU_UW_root, 'images', 'form finding animations', f'form_finding_{model_name}_{0}.mp4'), writer=writer)

Flipping edge 2, 6, and 12 would make the network smoother

In [68]:
# net.flip_curve(2, n = 10)          # Flip the curvature of one edge
net.auto_flip_curves(n = 10)        # Automatically flip the curvature of the edges. Directions will become 1, -1, 1, -1, ...
net.flip_curves()                   # Flip all the edges.
net.flip_curves([2,5], n = 10)       # Flip the curvature of multiple edges
net.net_plot(color=True, plot_type='arcs', elables=True)

In [69]:
net.vertices_scaled/10, R/10

(array([[-9.56822955,  0.1599447 ,  0.        ],
        [-2.98725789,  1.02594216,  0.        ],
        [ 2.22896803, -0.04942054,  0.        ],
        [ 3.47036802, -0.12901583,  0.        ],
        [ 9.83149805, -0.02044095,  0.        ],
        [ 0.04213614, -9.92972345,  0.        ],
        [ 0.22024153, -3.92478708,  0.        ],
        [-0.90033299,  3.14909649,  0.        ],
        [-0.13545229,  9.60615954,  0.        ]]),
 array([2.11284741e+06, 8.46968548e+00, 9.99562412e-01, 2.02510550e+06,
        1.91227118e+06, 5.82011303e+00, 5.71052514e+00, 2.06971701e+06,
        1.09786690e+01, 1.67501311e+01, 1.73906321e+06, 5.98161140e+00]))

### Step 4: Account for intersections. 
Curved paths and intersections are automatically set up in the backend

In [70]:
printing_params = load_params(r'DATA/NT_Eel_0.2mm_og.json')
# When no interpolation function is provided, linear interpolation is used. No other interpolation function is implemented (yet?).
# net.jump_at_intersection(intersection_width = 2, intersection_height = 2, interpolation_function=None) # These settings are just for visibility, this would be way to much
# These setting are more appropriate for printing
net.jump_at_intersection(intersection_width = printing_params['d_nozzle']*1.5, intersection_height = printing_params['layer_height'], interpolation_function=None) 
net.net_plot(color=True, plot_type='arcs', elables=True)

Add loops to easily attach the printed network to a frame

In [71]:
alpha_loop = np.deg2rad(40) # The angle of the loop
L_loop  = 4                # The length of the loop
n_points = 60               # The number of points in the loop

start_loop_bools = [True, True, False]
end_loop_bools = [True, True, False]

net.all_loop_to_path(start_loop_bools, end_loop_bools, L_loop, alpha_loop, n_points)

Sometimes it takes a while before material starts extruding. This should preferably be fixed in the printer set up, but can me forced by adding a running start to a path

In [72]:
start_loop_bools = [False, False, False]
end_loop_bools = [False, False, False]
L_loop  = 2
n_points = 5
net.add_running_start(start_loop_bools, end_loop_bools, L_loop, n_points)

In [73]:
net.save_network(os.path.join(BYU_UW_root, 'networks', model_name + '_net.pkl'))

### Step 5: Generate G-code
Import and finalize printing parameters for materials and starting and end .gcode for the top. The start gcode has additional settings for temperatures and Linear Advance (K-factor). Linear advance is important for TPU as it is very visco-elastic. Linear advance can account for this

In [88]:
printing_params = load_params(r'DATA/NT_Eel_0.2mm_og.json')
start_gcode     = open(r'DATA/start_gcode.gcode', 'r').read()
end_gcode       = open(r'DATA/end_gcode.gcode', 'r').read()

temperature_settings = {'first_layer_bed_temperature': 65, 'first_layer_temperature':205, 'K-factor': 0.20}
g_code = replace_brackets(start_gcode, temperature_settings)

comment = ''

bed_width = 230
bed_height = 210

gen = G_code_generator(printing_params=printing_params)

printing_params['print_feedrate']

15.0

Make a Purge Line

In [89]:
point0, point1 = [20,15,0.3], [90,15,0.3]
point2, point3 = [90,10,0.3], [20,10,0.3]
g_code += gen.move_to_point(point0[0:2], point0[2] + printing_params['nozzle_lift'], comment='Move to start point')
g_code += gen.move_to_point(point0[0:2], point0[2], comment='Lower Nozzle')
g_code += gen.unretract()
g_code += gen._print_line(
        point0=point0,
        point1=point1,
        move_to_start=False, # move to start point without extruding
        extrude_factor=printing_params['extrude_factor']*2.5,
        comment=comment)
g_code += gen.retract()
g_code += gen.move_to_point(point2[0:2], point2[2] + printing_params['nozzle_lift'], comment='Move to start point')
g_code += gen.move_to_point(point2[0:2], point2[2], comment='Lower Nozzle')
g_code += gen.unretract()
g_code += gen._print_line(
        point0=point2,
        point1=point3,
        move_to_start=False, # move to start point without extruding
        extrude_factor=printing_params['extrude_factor']*2.5,
        comment=comment)
g_code += gen.retract()
g_code += gen.wipe(2 * np.pi) # Wipe the nozzle horizontally

Make the paths

In [90]:
for path_i, cor_list in enumerate(net.paths_xyz):
    g_code += '\n;Path (' + str(path_i) + ' ' + str(path_i) + ')\n'
    cor_list   = np.array(cor_list)
    # Move the coordinates to the center of the bed and add the layer height
    cor_list[:,0] += bed_width/2 -14
    cor_list[:,1] += bed_height/2 
    cor_list[:,2] += printing_params['layer_height']
    g_code += '\n'
    # Move the coordinates to the start point
    g_code += gen.move_to_point(cor_list[0][0:2], cor_list[0][2] + printing_params['nozzle_lift'], comment='Move to start point')
    # Lower the nozzle
    g_code += gen.move_to_point(cor_list[0][0:2], cor_list[0][2], comment='Lower Nozzle')
    # Unretract the filament
    g_code += gen.unretract()
    # Print the path
    for point0, point1 in zip(cor_list[:-1], cor_list[1:]):
        g_code += gen._print_line(
            point0=point0,
            point1=point1,
            move_to_start=False, # move to start point without extruding
            extrude_factor = printing_params['extrude_factor'],
            comment=comment)
    # Retract the filament
    g_code += gen.retract()
    # Wipe the nozzle
    g_code += gen.wipe_from_last_points(g_code)    
    # Raise the nozzle
    g_code += gen.move_to_point(point1[0:2], point1[2] + printing_params['nozzle_lift'], speed_factor=0.5, comment='Raise Nozzle')

g_code += end_gcode
with open(os.path.join('DATA', 'generated_gcodes', model_name + '.gcode'), "w") as g_code_file:
    g_code_file.write(g_code)
print('G-code generated')

G-code generated


In [91]:
minx, miny = 90, 90
maxx, maxy = 0, 0
for path_i, cor_list in enumerate(net.paths_xyz):
    cor_list   = np.array(cor_list)
    cor_list[:,0] += bed_width/2 -14
    cor_list[:,1] += bed_height/2 
    minx = min(minx, np.min(cor_list[:,0]))
    miny = min(miny, np.min(cor_list[:,1]))
    maxx = max(maxx, np.max(cor_list[:,0]))
    maxy = max(maxy, np.max(cor_list[:,1]))

minx , miny , maxx , maxy 

(-0.010433801841600143,
 0.34309684596132684,
 204.6747371649975,
 206.3968149183594)